## Moat Brand Reporting Refresh

In [1]:
%load_ext autoreload

In [2]:
%autoreload

from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings
from rtf_utils.bq_schema import moat_schema_dict

warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
import json

In [3]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")
moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')
print("Token Loaded")

Creds Loaded
Token Loaded


In [5]:
start_date = '20191001'
end_date = '20191006'

In [1]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

NameError: name 'json' is not defined

In [9]:
Q4 = [(2506,{'level1':22783112},['date','level1','level3']), ## disp stadia,
      (2506,{'level1':23221764},['date','level1','level3']), ## disp nest 1,
      (2506,{'level1':23197607},['date','level1','level3']), ## disp nest 2
      (2506,{'level1':23278016},['date','level1','level3']), ## disp assistant
      (2698,{'level1':22783112},['date','level1','level3']), ## vid stadia,
      (2698,{'level1':23221764},['date','level1','level3']), ## vid nest 1,
      (2698,{'level1':23197607},['date','level1','level3']), ## vid nest 2,
      (2698,{'level1':23278016},['date','level1','level3']),
      (6179366,{'level1':2604566125},['date','level1','level4']),## yt res stadia
      (6196284,{'level1':1533586},['date','level1','level4']) ## yt res dbm assistant

     
     ]


In [7]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id,level_filters,dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    blob = gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    return blob

# Prototype WorkFlow

In [22]:
tile_id, level_filter, dimensions = (13120,{'level1':10154328017481183},['date','level1','level4'])

In [23]:
filename = get_moat_data(tile_id,'20190801','20190901',level_filter,dimensions)

Data Available, Storing


In [ ]:
filename

In [9]:
b = upload_local_file(filename,"rtf_staging",folder="brand_reporting")

In [10]:
import os
import time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
#for x in bonito:
for i,x in enumerate(Q4):
    time.sleep(10)
    tile_id, level_filter, dimensions = x
    print(tile_id, level_filter, dimensions)
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    print("Stored at {}".format(filename))    
    try:
        file_uri = upload_local_file(filename,"rtf_staging",folder="brand_reporting")
        file_uri = "gs://rtf_staging/" + file_uri.name
    
        print("Load Schema")
        #schema_dict = moat_schema_dict[tile_id]

        dest_table = "Moat_{}_{}".format(tile_id,i)

        print("Try BQ Upload: {}".format(dest_table))

        bq = BigQuery(credentialsFromVault)
        resp = bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table,
                        #schema=schema_dict['schema'],
                        
                        extension='json')
    except Exception as e:
        print(e)
        continue

2506 {'level1': 22783112} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2506_22783112.json
Load Schema
Try BQ Upload: Moat_2506_0
2506 {'level1': 23221764} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2506_23221764.json
Load Schema
Try BQ Upload: Moat_2506_1
2506 {'level1': 23197607} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2506_23197607.json
Load Schema
Try BQ Upload: Moat_2506_2
2506 {'level1': 23278016} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2506_23278016.json
Load Schema
Try BQ Upload: Moat_2506_3
2698 {'level1': 22783112} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2698_22783112.json
[Errno 2] No such file or directory: 'Moat_2698_22783112.json'
2698 {'level1': 23221764} ['date', 'level1', 'level3']
Data Available, Storing
Stored at Moat_2698_23221764.json
[Errno 2] No such file or directory: 'Moat_2698_23221764.json'
2698 {'level1': 23197607} ['date', 'leve

AttributeError: 'NoneType' object has no attribute 'get'

In [13]:
import requests
auth_header = 'Bearer {}'.format(moat_token)

fields = ["impressions_analyzed",
                    "loads_unfiltered",
                    "susp_human",
                    "human_and_viewable",
                    ] 

query = {
                'metrics': ','.join(['level1']),
                'start': start_date,
                'end': end_date,
                'brandId':13120, ## this is the tile ID 
                } 


resp = requests.get('https://api.moat.com/1/stats.json',
                        params=query,
                        headers={'Authorization': auth_header,
                                    'User-agent': 'Essence Global 1.0'}
                       )
r = resp.json()

In [ ]:
f